In [1]:
import numpy as np
import open3d as o3d
import copy
from matplotlib import pyplot as plt

In [2]:
from readData import *
from showFig import *
from showCloud import *
from warp import *

In [3]:
def draw_transform(src, tgt, T):
    srct = copy.deepcopy(src)
    tgtt = copy.deepcopy(tgt)
    srct.paint_uniform_color([1, 0.5, 0])
    tgtt.paint_uniform_color([0, 0.5, 0.9])
    srct.transform(T)
    o3d.visualization.draw_geometries([srct, tgtt])
    return src.transform(T)

In [4]:
def preprocess(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)
    radius_normal = voxel_size * 2
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30)
    )

    radius_feature = voxel_size * 50
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100)
    )
    print(pcd_fpfh.data.mean())
    # print(pcd_fpfh.data.max())
    return pcd_down, pcd_fpfh 

In [5]:
def showFeature(pcd, pcdf, eps=0.6):
    scale = np.linalg.norm(pcdf.data, axis=0)
    # plt.plot(list(range(len(scale0))), scale0)
    # plt.show()
    pcdc = np.hstack([np.asarray(pcd.points), scale.reshape(-1,1)])
    pcdc = pcdc[(scale>=300) & (scale<=350)]
    pcdc[:,3] = 1-(pcdc[:,3]-pcdc[:,3].min())/(pcdc[:,3].max()-pcdc[:,3].min())
    # pcdc[:,3][pcdc[:,3]<eps] = 0 
    print(pcdc.shape)
    return pcdc

In [6]:
def filterFeature(raw_pcd, raw_pcdf, eps=0.8):
    pcd = copy.deepcopy(raw_pcd)
    pcdf = copy.deepcopy(raw_pcdf)
    scale = np.linalg.norm(pcdf.data, axis=0)
    pcdc = np.hstack([np.asarray(pcd.points), pcdf.data.swapaxes(0,1), scale.reshape(-1,1)])
    pcdc = pcdc[(scale>=300) & (scale<=350)]
    pcdc[:,-1] = 1-(pcdc[:,-1]-pcdc[:,-1].min())/(pcdc[:,-1].max()-pcdc[:,-1].min())
    pcdc = pcdc[pcdc[:,-1]>eps]
    pcd.points = o3d.utility.Vector3dVector(pcdc[:, :3])
    pcdf.data = pcdc[:, 3:3+33].swapaxes(0,1)
    # print(feature.shape)
    return pcd, pcdf

In [7]:
points0, mat0 = getPoints("..\dataset\scan0.npy", prange=(200, 150, (-100, -30)), skip=20)
points1, mat1 = getPoints("..\dataset\scan1.npy", prange=(200, 150, (-100, -30)), skip=20)

read in ..\dataset\scan0.npy
x range: -197 ~ 198
y range: -153 ~ 145
z range: -66 ~ 0
read in ..\dataset\scan1.npy
x range: -197 ~ 198
y range: -158 ~ 145
z range: -69 ~ 0


In [8]:
voxel_size = 0.1
src = o3d.geometry.PointCloud()
src.points = o3d.utility.Vector3dVector(points0)
tgt = o3d.geometry.PointCloud()
tgt.points = o3d.utility.Vector3dVector(points1)
# o3d.visualization.draw_geometries([src, tgt])
srcd, srcf = preprocess(src, voxel_size)
tgtd, tgtf = preprocess(tgt, voxel_size)

src_sparse, srcf = filterFeature(srcd, srcf)
tgt_sparse, tgtf = filterFeature(tgtd, tgtf)
# print(len(src.points))
# print(srcf.data.shape)

18.181818181818183
18.181305223201118


In [9]:
result = o3d.pipelines.registration.registration_fast_based_on_feature_matching(
    src_sparse, tgt_sparse,
    srcf, tgtf
)
draw_transform(srcd, tgtd, result.transformation)

PointCloud with 35414 points.

In [10]:
draw_transform(src_sparse, tgt_sparse, result.transformation)

PointCloud with 1008 points.

In [15]:
srcc = showFeature(srcd, srcf)
tgtc = showFeature(tgtd, tgtf)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 35414 and the array at index 1 has size 1008

In [54]:
fig = dynVisual([srcc, tgtc], ["src", "tgt"], 0.5, 4)
fig.write_html("cloudpoint.html")

In [55]:
fig = dynVisual([srcc], ["src"], 0.5, 4)
fig.write_html("cloudsrc.html")

In [56]:
fig = dynVisual([tgtc], ["tgt"], 0.5, 4)
fig.write_html("cloudtgt.html")

In [16]:
srcf.data.shape

(33, 35414)

In [17]:
src.points

std::vector<Eigen::Vector3d> with 35414 elements.
Use numpy.asarray() to access data.